In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, BatchNormalization
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras import optimizers
import numpy as np

### Loading CIFAR-10 dataset

In [2]:
(trainX, trainy), (testX, testy) = cifar10.load_data()

In [3]:
# print to make sure we have the correct shapes + number of images for training
print("number of train pictures:", trainX.shape)
print("number of trained picture values:", trainy.shape)
# divide by 255 to make [0,255] into [0,1] + print to make sure!
trainy = tf.keras.utils.to_categorical(trainy,10)
testy = tf.keras.utils.to_categorical(testy,10)
trainX = trainX/255.0
testX = testX/255.0

number of train pictures: (50000, 32, 32, 3)
number of trained picture values: (50000, 1)


### VGG-16 Clone Without Quantization

In [5]:
model_wq = tf.keras.Sequential()
#block-1
model_wq.add(Conv2D(input_shape=(32,32,3),
                    filters=64,kernel_size=(3,3),
                    padding="same", 
                    activation="relu",
                    name='block1_conv1'))
model_wq.add(BatchNormalization())
model_wq.add(Dropout(0.3))

model_wq.add(Conv2D(filters=64,
                    kernel_size=(3,3),
                    padding="same", 
                    activation="relu",
                    name='block1_conv2'))
model_wq.add(BatchNormalization())

model_wq.add(MaxPool2D(pool_size=(2,2), strides=(2,2), name='block1_pool'))


#block-2
model_wq.add(Conv2D(filters=128, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block2_conv1'))
model_wq.add(BatchNormalization())
model_wq.add(Dropout(0.4))

model_wq.add(Conv2D(filters=128, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block2_conv2'))
model_wq.add(BatchNormalization())

model_wq.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='block2_pool'))

#block-3
model_wq.add(Conv2D(filters=256, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block3_conv1'))
model_wq.add(BatchNormalization())
model_wq.add(Dropout(0.4))

model_wq.add(Conv2D(filters=256, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block3_conv2'))
model_wq.add(BatchNormalization())
model_wq.add(Dropout(0.4))

model_wq.add(Conv2D(filters=256, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block3_conv3'))
model_wq.add(BatchNormalization())

model_wq.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='block3_pool'))

#block-4
model_wq.add(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block4_conv1'))
model_wq.add(BatchNormalization())
model_wq.add(Dropout(0.4))

model_wq.add(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block4_conv2'))
model_wq.add(BatchNormalization())
model_wq.add(Dropout(0.4))

model_wq.add(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block4_conv3'))
model_wq.add(BatchNormalization())

model_wq.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='block4_pool'))

#block-5
model_wq.add(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block5_conv1'))
model_wq.add(BatchNormalization())
model_wq.add(Dropout(0.4))

model_wq.add(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block5_conv2'))
model_wq.add(BatchNormalization())
model_wq.add(Dropout(0.4))

model_wq.add(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block5_conv3'))
model_wq.add(BatchNormalization())

model_wq.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='block5_pool'))


#fc1, fc2 and predictions
model_wq.add(Dropout(0.5))
model_wq.add(Flatten(name='flatten'))
model_wq.add(Dense(units=512,activation="relu",name='fc1'))
model_wq.add(BatchNormalization())

model_wq.add(Dropout(0.5))
model_wq.add(Dense(units=10, activation="softmax",name='predictions'))

model_wq.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_14 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_10 (Dropout)         (None, 32, 32, 64)        0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_15 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)      

### Tranining withgout quantization vgg16

In [6]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

opt = SGD(learning_rate=0.1, decay=1e-6, momentum=0.9, nesterov=True)
# Compile the model
model_wq.compile(optimizer=opt, loss=tf.keras.losses.categorical_crossentropy,metrics=['accuracy'])

# Fit data to model
model_wq.fit(trainX, trainy,
          batch_size=50,
          epochs=100,
          verbose=1,
          validation_split=0.2)
model.save_weights('cifar10vgg_nonquant.h5')

Epoch 1/100
800/800 [==============================] - 13s 12ms/step - loss: 2.6187 - accuracy: 0.1730 - val_loss: 6.4007 - val_accuracy: 0.0971
Epoch 2/100
800/800 [==============================] - 10s 12ms/step - loss: 1.9199 - accuracy: 0.2528 - val_loss: 6.2614 - val_accuracy: 0.1107
Epoch 3/100
800/800 [==============================] - 10s 12ms/step - loss: 1.8065 - accuracy: 0.2902 - val_loss: 2.9115 - val_accuracy: 0.1690
Epoch 4/100
800/800 [==============================] - 10s 12ms/step - loss: 1.7293 - accuracy: 0.3243 - val_loss: 2.1482 - val_accuracy: 0.2536
Epoch 5/100
800/800 [==============================] - 10s 12ms/step - loss: 1.6506 - accuracy: 0.3663 - val_loss: 2.3405 - val_accuracy: 0.2739
Epoch 6/100
800/800 [==============================] - 10s 12ms/step - loss: 1.5854 - accuracy: 0.4024 - val_loss: 2.5533 - val_accuracy: 0.2721
Epoch 7/100
800/800 [==============================] - 10s 12ms/step - loss: 1.4847 - accuracy: 0.4517 - val_loss: 2.0361 - val_ac

NameError: name 'model' is not defined

In [7]:
model_wq.save_weights('cifar10vgg_nonquant.h5')

### Loss and Accuracy without quantization

In [8]:
score = model_wq.evaluate(testX, testy, verbose=1)
print("Test loss {:.4f}, accuracy {:.2f}%".format(score[0], score[1] * 100))

313/313 [==============================] - 2s 5ms/step - loss: 0.7216 - accuracy: 0.8418
Test loss 0.7216, accuracy 84.18%


### Quantization vs Without quantization Test loss
#### note that the model without quantization is 528 MB and the quantized model is 32 MB (16x decrease)

In [13]:
import tempfile
# Create float TFLite model.
model_wq = tf.lite.TFLiteConverter.from_keras_model(model)
model_wq = model_wq.convert()
print('converted')
# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')
print('files')
with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model)
print('done')
with open(float_file, 'wb') as f:
  f.write(model_wq)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

INFO:tensorflow:Assets written to: /var/folders/1q/ns7tqnvx55lfc3nl7vf90tth0000gn/T/tmpdricefb_/assets


INFO:tensorflow:Assets written to: /var/folders/1q/ns7tqnvx55lfc3nl7vf90tth0000gn/T/tmpdricefb_/assets


converted
files
done
Float model in Mb: 527.8010864257812
Quantized model in Mb: 32.207611083984375


## Save quantized model to upload on microcontroller

In [20]:
open("cifar_quant.tflite", "wb").write(quantized_tflite_model)

33772128